In [ ]:
import polars as pl
from datetime import date
import toml
from googleapiclient.discovery import build
from utils import auth, drive

In [ ]:
INPUT_FILE = 'january2024_mandatory_use_full'   # update this before running

In [ ]:
with open('../secrets.toml', 'r') as f:
    secrets = toml.load(f)

creds = auth.auth()
service = build('drive', 'v3', credentials=creds)

In [ ]:
no_violation = (
    drive.lazyframe_from_id_and_sheetname(
        service=service, file_id=secrets['files']['not_violation'], sheet_name='no_violation'
    )
    .with_columns(
        pl.col('exclude until').str.to_date(format='%m-%d-%y'),
        pl.col('final_id').cast(pl.String)
    )
    .filter(
        pl.col('exclude until') > date.today()
    )
)

In [ ]:
file_path = f'data/{INPUT_FILE}_nv.csv'
mu = (
    pl.scan_csv(f'data/{INPUT_FILE}.csv')
    .join(no_violation, on='final_id', how='anti')
    .collect()
    .write_csv(file_path)
)

print(f'{file_path} generated')